# Build123d Introductory Examples

The examples are taken from the [official documentation](https://build123d.readthedocs.io/en/latest/introductory_examples.html). Only the Builder mode code is implemented here.

In [10]:
import build123d as bd
from ocp_vscode import show

## 1. Simple Rectangular Plate

In [ ]:
length, width, thickness = 80.0, 60.0, 10.0

with bd.BuildPart() as ex1:
    bd.Box(length, width, thickness)

show(ex1)

Using port 3939
Jupyter kernel running
Jupyter connection file path written to /home/vscode/.ocpvscode
+


## 2. Plate with Hole

In [ ]:
length, width, thickness = 80.0, 60.0, 10.0
center_hole_dia = 22.0

with bd.BuildPart() as ex2:
    bd.Box(length, width, thickness)
    bd.Cylinder(radius=center_hole_dia / 2, height=thickness,
                mode=bd.Mode.SUBTRACT)

show(ex2)

Using port 3939
Jupyter kernel running
Jupyter connection file path written to /home/vscode/.ocpvscode
+


## 3. Extruded prismatic solid

In [ ]:
length, width, thickness = 80.0, 60.0, 10.0

with bd.BuildPart() as ex3:
    with bd.BuildSketch() as ex3_sk:
        bd.Circle(width)
        bd.Rectangle(length / 2, width / 2,
                     mode=bd.Mode.SUBTRACT)
    bd.extrude(amount=2 * thickness)

show(ex3)

## 4. Building Profiles using lines and arcs

In [ ]:
length, width, thickness = 80.0, 60.0, 10.0

with bd.BuildPart() as ex4:
    with bd.BuildSketch() as ext4_sk:
        with bd.BuildLine() as ex4_ln:
            l1 = bd.Line((0, 0), (length, 0))
            l2 = bd.Line((length, 0), (length, width))
            l3 = bd.ThreePointArc((length, width), (width, width*1.5), (0, width))
            # For tangent arc use the following:
            # l3 = ThreePointArc((length, width),
            #                    (length/2, width+length/2),
            #                    (0, width))
            l4 = bd.Line((0, width), (0, 0))
        bd.make_face()
    bd.extrude(amount=thickness)

show(ex4)

## 5. Moving the current working point

In [11]:
a, b, c, d = 90, 45, 15, 7.5

with bd.BuildPart() as ex5:
    with bd.BuildSketch() as ex5_sk:
        bd.Circle(a)
        with bd.Locations((b, 0.0)):
            bd.Rectangle(c, c, mode=bd.Mode.SUBTRACT)
        with bd.Locations((0, b)):
            bd.Circle(d, mode=bd.Mode.SUBTRACT)
    bd.extrude(amount=c)

show(ex5)

+


## 6. Using Points Lists

In [13]:
a, b, c = 80, 60, 10

with bd.BuildPart() as ex6:
    with bd.BuildSketch() as ex6_sk:
        bd.Circle(a)
        with bd.Locations((b, 0), (0, b), (-b, 0), (0, -b)):
            bd.Circle(c, mode=bd.Mode.SUBTRACT)
    bd.extrude(amount=c)

show(ex6)

c

## 7. Polygons

In [15]:
a, b, c = 60, 80, 5

with bd.BuildPart() as ex7:
    with bd.BuildSketch() as ex7_sk:
        bd.Rectangle(a, b)
        with bd.Locations((0, 3 * c), (0, -3 * c)):
            bd.RegularPolygon(radius=2 * c, side_count=6, mode=bd.Mode.SUBTRACT)
    bd.extrude(amount=c)

show(ex7)

c

## 8. Polylines

In [17]:
(L, H, W, t) = (100.0, 20.0, 20.0, 1.0)
pts = [
    (0, H / 2.0),
    (W / 2.0, H / 2.0),
    (W / 2.0, (H / 2.0 - t)),
    (t / 2.0, (H / 2.0 - t)),
    (t / 2.0, (t - H / 2.0)),
    (W / 2.0, (t - H / 2.0)),
    (W / 2.0, H / -2.0),
    (0, H / -2.0),
]

with bd.BuildPart() as ex8:
    with bd.BuildSketch(bd.Plane.YZ) as ex8_sk:
        with bd.BuildLine() as ex8_ln:
            bd.Polyline(pts)
            bd.mirror(ex8_ln.line, about=bd.Plane.YZ)
        bd.make_face()
    bd.extrude(amount=L)

show(ex8)

c


## 9. Selectors, Fillets and Chamfers

In [21]:
length, width, thickness = 80.0, 60.0, 10.0

with bd.BuildPart() as ex9:
    bd.Box(length, width, thickness)
    bd.chamfer(ex9.edges().group_by(bd.Axis.Z)[-1], length=4)
    bd.fillet(ex9.edges().filter_by(bd.Axis.Z), radius=5)

show(ex9)

c


## 10. Select Last and Hole

In [23]:
length, width, thickness = 80.0, 60.0, 10.0

with bd.BuildPart() as ex10:
    bd.Box(length, width, thickness)
    bd.Hole(radius=width / 4)
    bd.fillet(ex10.edges(bd.Select.LAST).group_by(bd.Axis.Z)[-1], radius=2)

show(ex10)

c

## 11. Use Face as Plane for BuildSketch and introduce GridLocations

In [34]:
length, width, thickness = 80.0, 60.0, 10.0

with bd.BuildPart() as ex11:
    bd.Box(length, width, thickness)
    bd.chamfer(ex11.edges().group_by(bd.Axis.Z)[-1], length=4)
    bd.fillet(ex11.edges().filter_by(bd.Axis.Z), radius=5)
    bd.Hole(radius=width / 4)
    bd.fillet(ex11.edges(bd.Select.LAST).sort_by(bd.Axis.Z)[-1], radius=2)
    with bd.BuildSketch(ex11.faces().sort_by(bd.Axis.Z)[-1]) as ex11_sk:
        with bd.GridLocations(length / 2, width / 2, 2, 2):
            bd.RegularPolygon(radius=5, side_count=5)
    bd.extrude(amount=-thickness, mode=bd.Mode.SUBTRACT)

show(ex11)

c


## 12. Define Edge with a Spline

In [31]:
pts = [
    (55, 30),
    (50, 35),
    (40, 30),
    (30, 20),
    (20, 25),
    (10, 20),
    (0, 20),
]

with bd.BuildPart() as ex12:
    with bd.BuildSketch() as ex12_sk:
        with bd.BuildLine() as ex12_ln:
            l1 = bd.Spline(pts)
            l2 = bd.Line((55, 30), (60, 0))
            l3 = bd.Line((60, 0), (0, 0))
            l4 = bd.Line((0, 0), (0, 20))
        bd.make_face()
    bd.extrude(amount=10)

show(ex12)

+


## 13. CounterBoreHoles, CounterSinkHoles and PolarLocations

In [35]:
a, b = 40, 4

with bd.BuildPart() as ex13:
    bd.Cylinder(radius=50, height=10)
    with bd.Locations(ex13.faces().sort_by(bd.Axis.Z)[-1]):
        with bd.PolarLocations(radius=a, count=4):
            bd.CounterSinkHole(radius=b, counter_sink_radius=2 * b)
        with bd.PolarLocations(radius=a, count=4,
                               start_angle=45, angular_range=360):
            bd.CounterBoreHole(radius=b,
                               counter_bore_radius=2 * b,
                               counter_bore_depth=b)

show(ex13)

+


## 14. Position on a Line with '@', '%' and introduce Sweep

## 15. Mirroring Symmetric Geometry